In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [2]:
# Install PyTorch Geometric and dependencies for PyTorch 2.0.1 + CUDA 11.8
!pip install torch==2.0.1+cu118 torchvision==0.15.2+cu118 torchaudio==2.0.2 --index-url https://download.pytorch.org/whl/cu118
!pip install torch-geometric torch-scatter torch-sparse torch-cluster torch-spline-conv -f https://data.pyg.org/whl/torch-2.0.1+cu118.html


Looking in indexes: https://download.pytorch.org/whl/cu118
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.3/2.3 GB 987.2 kB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 6.1/6.1 MB 124.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 4.4/4.4 MB 117.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 63.3/63.3 MB 39.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 132.3/132.3 kB 12.0 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Created wheel for lit: filename=lit-15.0.7-py3-none-any.whl size=89990 sha256=63cd5094eb4c51dd752b4537c78644f1fff2069c5ccfa3f094c5dbcca40cbbfd
  Stored in directory: /root/.cache/pip/wheels/fc/5d/45/34fe9945d5e45e261134e72284395be36c2d4828af38e2b0fe
Successfully built lit
  Attempting uninstall: triton
    Found existing installation: triton 3.2.0
    Uninstalling triton-3.2.0:
      Successfully uninstalled triton-3.2.0
  Attempting uninstall: torch
    Found existing installation: torch 2.

In [3]:
#Setting up paths

import os
import torch
import pandas as pd
from tqdm import tqdm
from torch_geometric.data import Data

#Check device
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
print(f"Using Device: {device}")

#Define paths
project_root = "/content/drive/MyDrive/Rebuilding_and_Modifying_GraphDTA"
data_path = f"{project_root}/data"

affinity_df = pd.read_csv(f"{data_path}/kiba_affinity_df.csv")
print(affinity_df.head())


A module that was compiled using NumPy 1.x cannot be run in
NumPy 2.0.2 as it may crash. To support both 1.x and 2.x
versions of NumPy, modules must be compiled with NumPy 2.0.
Some module may need to rebuild instead e.g. with 'pybind11>=2.12'.

If you are a user of the module, the easiest solution will be to
downgrade to 'numpy<2' or try to upgrade the affected module.
We expect that some modules will need time to support NumPy 2.

Traceback (most recent call last):  File "<frozen runpy>", line 198, in _run_module_as_main
  File "<frozen runpy>", line 88, in _run_code
  File "/usr/local/lib/python3.11/dist-packages/colab_kernel_launcher.py", line 37, in <module>
    ColabKernelApp.launch_instance()
  File "/usr/local/lib/python3.11/dist-packages/traitlets/config/application.py", line 992, in launch_instance
    app.start()
  File "/usr/local/lib/python3.11/dist-packages/ipykernel/kernelapp.py", line 712, in start
    self.io_loop.start()
  File "/usr/local/lib/python3.11/dist-package

Using Device: cuda
   Drug_Index  Protein_Index  \
0           0              0   
1           0              1   
2           0              2   
3           0              3   
4           0              4   

                                              SMILES  \
0  COC1=C(C=C2C(=C1)N=CN2C3=CC(=C(S3)C#N)OCC4=CC=...   
1  COC1=C(C=C2C(=C1)N=CN2C3=CC(=C(S3)C#N)OCC4=CC=...   
2  COC1=C(C=C2C(=C1)N=CN2C3=CC(=C(S3)C#N)OCC4=CC=...   
3  COC1=C(C=C2C(=C1)N=CN2C3=CC(=C(S3)C#N)OCC4=CC=...   
4  COC1=C(C=C2C(=C1)N=CN2C3=CC(=C(S3)C#N)OCC4=CC=...   

                                            Sequence  KIBA_Score  
0  MSAAVTAGKLARAPADPGKAGVPGVAAPGAPAAAPPAKEIPEVLVD...         0.0  
1  MRPSGTAGAALLALLAALCPASRALEEKKVCQGTSNKLTQLGTFED...         1.1  
2  MELAALCRWGLLLALLPPGAASTQVCTGTDMKLRLPASPETHLDML...         1.1  
3  MSGGGPSGGGPGGSGRARTSSFAEPGGGGGGGGGGPGGSASGPGGT...         1.3  
4  MSGRPRTTSFAESCKPVQQPSAFGSMKVSRDKDGSKVTTVVATPGQ...         1.3  


In [4]:
#Load drug graph tensors
import torch


drug_graphs = torch.load(f"{data_path}/kiba_drug_graphs.pt", weights_only=False)


#Load protein sequences

protein_seq = torch.load(f"{data_path}/kiba_protein_seqs.pt", weights_only=False)

In [5]:
from torch.utils.data import Dataset

class DTADataset(Dataset):
    def __init__(self, affinity_df, drug_graphs, protein_sequences):
        self.data = []
        for _, row in affinity_df.iterrows():
            d_idx = row["Drug_Index"]
            p_idx = row["Protein_Index"]
            y = row["KIBA_Score"]

            if d_idx in drug_graphs and p_idx in protein_sequences:
                drug_graph = drug_graphs[d_idx]
                protein_seq = protein_sequences[p_idx]
                self.data.append((drug_graph, protein_seq, torch.tensor([y], dtype=torch.float)))

    def __len__(self):
        return len(self.data)

    def __getitem__(self, idx):
        return self.data[idx]

In [6]:
#Build collate function

from torch.nn.utils.rnn import pad_sequence
from torch_geometric.data import Batch

def collate_fn(batch):
  drug_graphs, protein_seqs, labels = zip(*batch)
  drug_batch = Batch.from_data_list(drug_graphs)
  padded_proteins = pad_sequence(protein_seqs, batch_first=True, padding_value = 0)
  labels = torch.stack(labels)
  return drug_batch, padded_proteins, labels

In [7]:
#Create train/val/test and loaders

from torch.utils.data import DataLoader, random_split

full_dataset = DTADataset(affinity_df, drug_graphs, protein_seq)
train_size = int(0.8*len(full_dataset))
val_size = int(0.1*len(full_dataset))
test_size = len(full_dataset) - train_size - val_size

train_set, val_set, test_set = random_split(full_dataset, [train_size, val_size, test_size])

train_loader = DataLoader(train_set, batch_size = 512, shuffle=True, collate_fn=collate_fn)
val_loader = DataLoader(val_set, batch_size=512, collate_fn=collate_fn)
test_loader = DataLoader(test_set, batch_size=512, collate_fn=collate_fn)


In [8]:
#GraphDTA model  (GCN + 1D CNN)

import torch
import torch.nn as nn
import torch.nn.functional as F
from torch_geometric.nn import GCNConv, global_max_pool

class GraphDTA(nn.Module):
  def __init__(self, num_atom_features=83, protein_vocab_size=21, protein_embed_dim=128, out_channel=128, kernel_size=8, drug_output_dim=128, protein_output_dim=128, fc_hidden_dims=[1024, 512], dropout_rate=0.2, output_dim=1):

    super(GraphDTA, self).__init__()

    #Drug encoder
    self.gcn1 = GCNConv(num_atom_features, num_atom_features*2)
    self.gcn2 = GCNConv(num_atom_features*2, num_atom_features*4)
    self.gcn3 = GCNConv(num_atom_features*4, num_atom_features*8)
    self.drug_fc1 = nn.Linear(num_atom_features*8, 1024)
    self.drug_fc2 = nn.Linear(1024, 512)
    self.drug_fc3 = nn.Linear(512, drug_output_dim)
    self.relu = nn.ReLU()
    self.dropout = nn.Dropout(p=dropout_rate)

    #Protein encoder
    self.protein_embedding = nn.Embedding(protein_vocab_size, protein_embed_dim, padding_idx=0)
    self.conv1_protein = nn.Conv1d(protein_embed_dim, out_channels=out_channel, kernel_size=kernel_size)
    self.conv2_protein = nn.Conv1d(out_channel, protein_output_dim, kernel_size=kernel_size)
    self.protein_pool = nn.AdaptiveMaxPool1d(1) #output will be [batch_size, protein_output_dim, 1]

    #Fusion layers (MLP)
    self.fc1_combined = nn.Linear(drug_output_dim+protein_output_dim, fc_hidden_dims[0])
    self.dropout1 = nn.Dropout(p=dropout_rate)
    self.fc2_combined = nn.Linear(fc_hidden_dims[0], fc_hidden_dims[1])
    self.dropout2 = nn.Dropout(p=dropout_rate)
    self.out = nn.Linear(fc_hidden_dims[1], output_dim)

    self.relu = nn.ReLU()
    self.dropout = nn.Dropout(dropout_rate) #General dropout rate

  def forward(self, drug_data, protein_seq):
    #Drug graph forward
    x_drug, edge_index_drug, batch_drug = drug_data.x, drug_data.edge_index, drug_data.batch
    x_drug = self.relu(self.gcn1(x_drug, edge_index_drug))
    x_drug = self.relu(self.gcn2(x_drug, edge_index_drug))
    x_drug = self.relu(self.gcn3(x_drug, edge_index_drug))
    drug_emb_pooled = global_max_pool(x_drug, batch_drug)

    drug_emb = self.relu(self.drug_fc1(drug_emb_pooled))
    drug_emb = self.dropout(drug_emb)
    drug_emb = self.relu(self.drug_fc2(drug_emb))
    drug_emb = self.dropout(drug_emb)
    drug_emb = self.relu(self.drug_fc3(drug_emb))
    drug_emb = self.dropout(drug_emb)

    #Protein sequence forward
    seq_emb = self.protein_embedding(protein_seq)
    seq_emb = seq_emb.permute(0, 2, 1)
    seq_conv = self.relu(self.conv1_protein(seq_emb))
    seq_conv = self.relu(self.conv2_protein(seq_conv))
    protein_emb = self.protein_pool(seq_conv).squeeze(-1)

    #Fusion
    combined_emb = torch.cat((drug_emb, protein_emb), dim = 1)
    x_combined = self.relu(self.fc1_combined(combined_emb))
    x_combined = self.dropout1(x_combined)
    x_combined = self.relu(self.fc2_combined(x_combined))
    x_combined = self.dropout2(x_combined)

    output = self.out(x_combined)
    if output.shape[-1] == 1:
      output = output.squeeze(-1)
    return output


In [9]:
#Define training functions
def train(model, device, loader, optimizer, criterion):
  model.train()
  total_loss = 0
  for drug_graph, protein_seq, affinity in loader:
    drug_graph = drug_graph.to(device)
    protein_seq = protein_seq.to(device)
    affinity = affinity.to(device).squeeze()

    optimizer.zero_grad()
    output = model(drug_graph, protein_seq)
    loss = criterion(output, affinity)
    loss.backward()
    optimizer.step()
    total_loss += loss.item()
    return total_loss / len(loader)


In [10]:
#Define RMSE (PyTorch)
def rmse_torch(pred, true):
    return torch.sqrt(torch.mean((pred - true) ** 2)).item()

#Define CI(PyTorch)
def concordance_index_torch(y_true, y_pred):
    """Returns CI (pure PyTorch)"""
    concordant = 0.0
    permissible = 0.0
    n = len(y_true)

    for i in range(n):
        for j in range(i + 1, n):
            if y_true[i] != y_true[j]:
                permissible += 1
                if (y_pred[i] - y_pred[j]) * (y_true[i] - y_true[j]) > 0:
                    concordant += 1
    return concordant / permissible if permissible != 0 else 0.0


#Define evaluation function
def evaluate(model, device, val_loader, loader):
  model.eval()
  y_pred, y_true = [], []
  total_val_loss = 0
  num_samples = 0
  with torch.no_grad():
    for drug_graph, protein_seq, affinity in val_loader:
      drug_graph = drug_graph.to(device)
      protein_seq = protein_seq.to(device)
      affinity_label_device = affinity.to(device).squeeze()

      output = model(drug_graph, protein_seq)

      loss = criterion(output, affinity_label_device)
      total_val_loss += loss.item() * affinity_label_device.size(0)
      num_samples += affinity_label_device.size(0)

      y_pred.extend(output.detach().cpu().tolist())
      y_true.extend(affinity.squeeze().tolist())

  y_pred_tensor = torch.tensor(y_pred)
  y_true_tensor = torch.tensor(y_true)

  avg_val_loss = total_val_loss / num_samples

  metrics =  {
        'val_loss': avg_val_loss,
        'rmse': rmse_torch(y_pred_tensor, y_true_tensor),
        'ci': concordance_index_torch(y_true, y_pred)
    }
  return metrics

In [11]:
#Initialize model, optimizer and loader

import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import DataLoader
from torch.utils.data import DataLoader

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

model = GraphDTA(num_atom_features=83, protein_vocab_size=21, protein_embed_dim=128, out_channel=128, kernel_size=8, drug_output_dim=128, protein_output_dim=128, fc_hidden_dims=[1024, 512], dropout_rate=0.2, output_dim=1).to(device)
criterion = nn.MSELoss()
optimizer = optim.Adam(model.parameters(), lr=0.0005)

In [12]:
#Train the model

for epoch in range(1, 501):
  train_loss = train(model, device, train_loader, optimizer, criterion)
  val_metrics = evaluate(model, device, val_loader, criterion)
  print(f"Epoch {epoch:02d} | Loss: {train_loss: .4f} | Val RMSE: {val_metrics['rmse']:.4f} | CI: {val_metrics['ci']:.4f}")

Epoch 01 | Loss:  0.7466 | Val RMSE: 11.2222 | CI: 0.5366
Epoch 02 | Loss:  0.6810 | Val RMSE: 10.5258 | CI: 0.5418
Epoch 03 | Loss:  0.5982 | Val RMSE: 9.4049 | CI: 0.5473
Epoch 04 | Loss:  0.4763 | Val RMSE: 7.5714 | CI: 0.5530
Epoch 05 | Loss:  0.3103 | Val RMSE: 4.6289 | CI: 0.5565
Epoch 06 | Loss:  0.1148 | Val RMSE: 0.9336 | CI: 0.5593
Epoch 07 | Loss:  0.0066 | Val RMSE: 6.2731 | CI: 0.5573
Epoch 08 | Loss:  0.2176 | Val RMSE: 5.3984 | CI: 0.5601
Epoch 09 | Loss:  0.1624 | Val RMSE: 2.7818 | CI: 0.5605
Epoch 10 | Loss:  0.0464 | Val RMSE: 0.8669 | CI: 0.5569
Epoch 11 | Loss:  0.0056 | Val RMSE: 2.0667 | CI: 0.5520
Epoch 12 | Loss:  0.0254 | Val RMSE: 3.1628 | CI: 0.5467
Epoch 13 | Loss:  0.0531 | Val RMSE: 3.7169 | CI: 0.5437
Epoch 14 | Loss:  0.0761 | Val RMSE: 3.8147 | CI: 0.5408
Epoch 15 | Loss:  0.0777 | Val RMSE: 3.5308 | CI: 0.5373
Epoch 16 | Loss:  0.0726 | Val RMSE: 2.8977 | CI: 0.5347
Epoch 17 | Loss:  0.0481 | Val RMSE: 1.9814 | CI: 0.5318
Epoch 18 | Loss:  0.0221 | Va

In [14]:
#Save model weights for fine tuning

model_path = f"{data_path}/kiba_graphdta_baseline_weights.pt"
torch.save(model.state_dict(), model_path)
print(f"Saved model_weights to {model_path}")

Saved model_weights to /content/drive/MyDrive/Rebuilding_and_Modifying_GraphDTA/data/kiba_graphdta_baseline_weights.pt


In [15]:
#Save model

full_model_path = f"{data_path}/kiba_graphdta_baseline_model.pt"
torch.save(model, full_model_path)
print(f"Saved model to {full_model_path}")

Saved model to /content/drive/MyDrive/Rebuilding_and_Modifying_GraphDTA/data/kiba_graphdta_baseline_model.pt
